### 705: 设计哈希集合

In [2]:
# 利用桶思想存储key值
class MyHashSet:

    def __init__(self):
        self.mod = 1000
        self.bucket = [[] for _ in range(self.mod)]

    def add(self, key: int) -> None:
        k = key % self.mod
        if key not in self.bucket[k]:
            self.bucket[k].append(key)

    def remove(self, key: int) -> None:
        k = key % self.mod
        if key in self.bucket[k]:
            self.bucket[k].remove(key)

    def contains(self, key: int) -> bool:
        k = key % self.mod
        return key in self.bucket[k]

### 706: 设计哈希映射

In [ ]:
# 同时存储key和value
class MyHashMap:
    def __init__(self):
        self.mod = 1000
        self.bucket = [[] for _ in range(self.mod)]

    def put(self, key: int, value: int) -> None:
        idx = key % self.mod
        for i in range (len(self.bucket[idx])):
            k,v = self.bucket[idx][i]
            if k == key:
                self.bucket[idx][i] = (key, value)
                return 
        self.bucket[idx].append((key, value))

    def get(self, key: int) -> int:
        idx = key % self.mod
        for (k,v) in self.bucket[idx]:
            if k == key:
                return v
        return -1

    def remove(self, key: int) -> None:
        idx = key % self.mod
        for (k,v) in self.bucket[idx]:
            if k == key:
                self.bucket[idx].remove((k,v))

### 146: LRU 缓存机制

In [ ]:
# 在Python中，有一种结合了哈希表与双向链表的数据结构OrderedDict
# move_to_end, popitem(last=False)
class LRUCache(collections.OrderedDict):

    def __init__(self, capacity: int):
        super().__init__()
        self.capacity = capacity

    def get(self, key: int) -> int:
        if key not in self:
            return -1
        self.move_to_end(key)
        return self[key]

    def put(self, key: int, value: int) -> None:
        if key in self:
            self.move_to_end(key)
        self[key] = value
        if len(self) > self.capacity:
            self.popitem(last=False)

# 需要用到一个哈希表和一个双向链表
    # 双向链表按照被使用的顺序存储了这些键值对，靠近头部的键值对是最近使用的，而靠近尾部的键值对是最久未使用的
    # 哈希表即为普通的哈希映射，通过缓存数据的键映射到其在双向链表中的位置
    # 在双向链表的实现中，使用一个伪头部/dummy head和伪尾部/dummy tail标记界限，这样在添加节点和删除节点的时候就不需要检查相邻的节点是否存在
# 定义字典存key和node(value)，定义伪头部和伪尾部，定义size记录现有容量
# moveToHead(node)(切断和接入), removeTail()
class DLinkedNode:
    def __init__(self, key=0, value=0):
        self.key = key
        self.value = value
        self.prev = None
        self.next = None

class LRUCache:

    def __init__(self, capacity: int):
        self.cache = dict()
        # 使用伪头部和伪尾部节点    
        self.head = DLinkedNode()
        self.tail = DLinkedNode()
        self.head.next = self.tail
        self.tail.prev = self.head
        self.capacity = capacity
        self.size = 0

    def get(self, key: int) -> int:
        if key not in self.cache:
            return -1
        # 如果 key 存在，先通过哈希表定位，再移到头部
        node = self.cache[key]
        self.moveToHead(node)
        return node.value

    def put(self, key: int, value: int) -> None:
        if key not in self.cache:
            # 如果 key 不存在，创建一个新的节点
            node = DLinkedNode(key, value)
            # 添加进哈希表
            self.cache[key] = node
            # 添加至双向链表的头部
            self.addToHead(node)
            self.size += 1
            if self.size > self.capacity:
                # 如果超出容量，删除双向链表的尾部节点
                removed = self.removeTail()
                # 删除哈希表中对应的项
                self.cache.pop(removed.key)
                self.size -= 1
        else:
            # 如果 key 存在，先通过哈希表定位，再修改 value，并移到头部
            node = self.cache[key]
            node.value = value
            self.moveToHead(node)

    def addToHead(self, node):
        node.prev = self.head
        node.next = self.head.next
        self.head.next.prev = node
        self.head.next = node

    def removeNode(self, node):
        node.prev.next = node.next
        node.next.prev = node.prev

    def moveToHead(self, node):
        self.removeNode(node)
        self.addToHead(node)

    def removeTail(self):
        node = self.tail.prev
        self.removeNode(node)
        return node

### 232: 用栈实现队列

In [ ]:
# 双栈
# 栈的 后进先出 特性相当于“逆转”了队列 先进先出 的效果，所以为了实现队列，我们需要使用两个栈，经过两次 “逆转” 满足队列的正常操作顺序
# 将所有元素都时刻保存在栈 b 中，也即任何操作过后，要保证全部元素保存在栈 b 中
class MyQueue:

    def __init__(self):
        """
        Initialize your data structure here.
        """
        self.a = []
        self.b = []


    def push(self, x: int) -> None:
        """
        Push element x to the back of queue.
        """
        while self.b:
            self.a.append(self.b.pop())
        self.a.append(x)
        while self.a:
            self.b.append(self.a.pop())


    def pop(self) -> int:
        """
        Removes the element from in front of queue and returns that element.
        """
        return self.b.pop()


    def peek(self) -> int:
        """
        Get the front element.
        """
        return self.b[-1] 


    def empty(self) -> bool:
        """
        Returns whether the queue is empty.
        """
        return len(self.b) == 0
